In [ ]:
# Calculate the correlations between the Law Dome ice core proxy records and SST values from ERSSTv5 using a multivariate linear regression
#
# This investigates increase in correlation or spatial skill by the inclusion of more species, and by smoothing the datasets

In [1]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 30.96 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39221,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 30.96 GiB
Comm: tcp://127.0.0.1:44665,Total threads: 4
Dashboard: http://127.0.0.1:40871/status,Memory: 6.19 GiB
Nanny: tcp://127.0.0.1:46065,


In [19]:
import numpy as np
import math
import xarray as xr


import matplotlib.pyplot as plt
import matplotlib.font_manager

matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
import cartopy.crs as ccrs

import datetime
today=datetime.date.today()

from arch.bootstrap import CircularBlockBootstrap
from scipy.stats import pearsonr
from scipy.ndimage import gaussian_filter
import pandas as pd

import matplotlib.ticker as ticker
plt.rcParams['font.family'] = 'Arial'
#plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'

In [74]:
#read in training data (ie, instrumental period.)


#read the ice core data
sigma=2.12 # for a 5-year width gaussian smooth

df=pd.read_csv('data/DSS_2k_winter_centred.csv')
df.rename(columns={'Year (CE)':'year','DJFMAM (μEq/L)':'djfmam','JJASON (μEq/L)':'jjason','Accumulation rate (IE m/y)':'accum'}, inplace=True)
df.set_index(['year'], inplace=True)
df=df.iloc[::-1]
icecoredata = xr.Dataset.from_dataframe(df)
icecoredata=icecoredata.assign_coords({'proxy':['djfmam','jjason','accum']})
icecoredata

<xarray.Dataset> Size: 195kB
Dimensions:             (year: 2028, proxy: 3)
Coordinates:
  * year                (year) int64 16kB -11 -10 -9 -8 ... 2013 2014 2015 2016
  * proxy               (proxy) <U6 72B 'djfmam' 'jjason' 'accum'
Data variables:
    Year BP1950         (year) int64 16kB 1961 1960 1959 1958 ... -64 -65 -66
    Sodium (μEq/L)      (year) float64 16kB 2.925 2.638 4.173 ... 3.275 4.326
    Chloride (μEq/L)    (year) float64 16kB 3.687 3.249 5.183 ... 3.847 4.994
    Magnesium (μEq/L)   (year) float64 16kB 0.46 0.455 0.73 ... 0.629 0.844
    Sulphate (μEq/L)    (year) float64 16kB 0.589 0.499 0.723 ... 0.724 0.795
    (d18O)              (year) float64 16kB -22.1 -22.08 ... -22.75 -22.51
    layer_thickness(m)  (year) float64 16kB 0.18 0.178 0.281 ... 0.692 0.7334
    accum               (year) float64 16kB 0.6621 0.6544 ... 0.6943 0.7346
    djfmam              (year) float64 16kB nan nan 0.373 ... -0.067 0.103 0.154
    jjason              (year) float64 16kB nan 0.543 0.757 ... 0.528 0.663 0.83
    DJFM (μEq/L)        (year) float64 16kB nan nan 0.1552 ... 0.04611 0.3387

In [76]:
#set up the ice core data with xarray
calibration_years=np.arange(1854,2017) #

sigma=2.12 #5 year smooth
smoothed_icecoredata_vals = gaussian_filter(icecoredata.values, sigma=sigma)
smoothed_icecoredata = xr.DataArray(smoothed_icecoredata_vals, coords=icecoredata.coords, dims=icecoredata.dims)

data_train=icecoredata[['year','djfmam','jjason','accum']].sel(dict(year=slice(1854,2017)))
data_train_smooth=smoothed_icecoredata[['year','djfmam','jjason','accum']].sel(dict(year=slice(1854,2017)))

data_train_smooth

data_full=icecoredata[['year','djfmam','jjason','accum']].sel(dict(year=slice(-9,2017)))

ValueError: different number of dimensions on data and dims: 0 vs 2

In [10]:

ss_jjason_unf=LD_ss_raw[-163:,2]
accum_unf=accum_raw[-163:,4]

ss_djfmam=gaussian_filter(LD_ss_raw[-163:,1],sigma) #warm season-sea salts
ss_jjason=gaussian_filter(LD_ss_raw[-163:,2],sigma) # cool season-sea salts


#ersst is the reconstruction target
ersst=xr.open_mfdataset('/home/lmjong/Datasets/ersst_v5/*.nc',parallel=True)
annAv_ersst=ersst['ssta'].groupby('time.year').mean(skipna=True)
annAv_ersst=annAv_ersst.fillna(0)

annAv_ersst=annAv_ersst.chunk(dict(year=-1))

annAv_ersst=annAv_ersst.isel(lev=0)
annAv_ersst.compute()

#calculate a few things for convenience
times=len(annAv_ersst.year)
nlats=len(annAv_ersst.lat)
nlons=len(annAv_ersst.lon)

h_scale=0.25
nyears=len(years)

lat = annAv_ersst['lat'].values
lon = annAv_ersst['lon'].values
sst = annAv_ersst.values

Exception ignored in: <function CachingFileManager.__del__ at 0x71aa59219ea0>
Traceback (most recent call last):
  File "/home/lmjong/miniconda3/envs/SST_reco/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/lmjong/miniconda3/envs/SST_reco/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID
Exception ignored in: <function CachingFileManager.__del__ at 0x71aa59219ea0>
Traceback (most recent call last):
  File "/home/lmjong/miniconda3/envs/SST_reco/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/lmjong

In [4]:
#create data structures for the training data
tmp=np.ones((len(years),3))
tmp[:,0]=ss_djfmam_unf.flatten()
tmp[:,1]=ss_jjason_unf.flatten()
tmp[:,2]=accum_unf.flatten()

data_train = xr.DataArray(tmp,
    coords={'year':years,'spec':['sss','wss','acc']})


data_train_sss=data_train.sel(spec=['sss'])
data_train_wss=data_train.sel(spec=['wss'])
data_train_sss_wss=data_train.sel(spec=['sss','wss'])
data_train_acc=data_train.sel(spec=['acc'])

NameError: name 'annAv_ersst' is not defined

In [11]:
# now create data structures for the full reconstruction
accum_long=accum_raw[1:,4]
long_years=np.arange(-10,2017)
tmp2=np.ones((len(long_years),3))
tmp2[:,0]=LD_ss_raw[:,1].flatten()
tmp2[:,1]=LD_ss_raw[:,2].flatten()
tmp2[:,2]=accum_long.flatten()

all_data = xr.DataArray(tmp2,
    coords={'time':long_years,'spec':['sss','wss','acc']}
)

# linearly interpolate to fill any gaps in the record so we don't have nans
all_data=all_data.interpolate_na(dim='time',max_gap=4,method="linear", fill_value="extrapolate")
all_data_sss=all_data.sel(spec=['sss'])
all_data_sss_wss=all_data.sel(spec=['sss','wss'])

In [12]:
from sklearn.linear_model import LinearRegression

def pearson(x,y):
    r,p=pearsonr(x,y)
    if math.isnan(r):
        r=0.0
    return r

In [13]:
def corr_only(train, ttarget):
    #corr_sss2[i]=pearsonr(x[:],ss_djfmam_unf)[0] This is what we are trying to achieve
    corr_coeff = pearson(ttarget, train)[0]
    return corr_coeff


def new_linreg(train, ttarget, full_data):
    # Wrapper around scipy linregress to use in apply_ufunc
    reg = LinearRegression()
    # print(target)

    model = reg.fit(train, ttarget)

    ypred = model.predict(train).flatten()
    corr_coeff = pearson(ttarget, train)[0]
    #need to adapt this and the return vals

    #bs = CircularBlockBootstrap(6, years,x=ttarget,y=ypred) #checked autocorrelation length of the data, 6 is consistant with block length chosen by Mudelsee code

    #if np.isnan(corr_coeff):
    #    cistmp=np.zeros(2)
    #else:
    #    cistmp=bs.conf_int(pearson,reps=2000,method='bca').flatten()

    ssta_recotmp = reg.predict(full_data).flatten()

    #return (ypred,corr_coeff,cistmp,ssta_recotmp)
    return (ypred, corr_coeff, ssta_recotmp)

In [14]:
corrs_sss=xr.apply_ufunc(corr_only,
                    data_train_sss,
                    annAv_ersst,
                    input_core_dims=[['year','spec'],['year']],
                 # output_core_dims=[[],[],[],[]],
                    #output_sizes={'bound':2},
                   # output_core_dims=[['year'],[],['bound'],['lyear']],
                    output_core_dims=[[]],
                    output_dtypes=[np.float32],
                    vectorize=True,
                    dask='parallelized',
                    )

corrs_wss=xr.apply_ufunc(corr_only,
                    data_train_wss,
                    annAv_ersst,
                    input_core_dims=[['year','spec'],['year']],
                 # output_core_dims=[[],[],[],[]],
                    #output_sizes={'bound':2},
                   # output_core_dims=[['year'],[],['bound'],['lyear']],
                    output_core_dims=[[]],
                    output_dtypes=[np.float32],
                    vectorize=True,
                    dask='parallelized',
                    )

corrs_acc=xr.apply_ufunc(corr_only,
                    data_train_acc,
                    annAv_ersst,
                    input_core_dims=[['year','spec'],['year']],
                 # output_core_dims=[[],[],[],[]],
                    #output_sizes={'bound':2},
                   # output_core_dims=[['year'],[],['bound'],['lyear']],
                    output_core_dims=[[]],
                    output_dtypes=[np.float32],
                    vectorize=True,
                    dask='parallelized',
                    )

NameError: name 'data_train_sss' is not defined

In [15]:
data_train_sss.squeeze('spec')
corrs_sss=xr.corr(data_train_sss.squeeze('spec'),annAv_ersst,dim='year')
corrs_sss.persist()

corrs_wss=xr.corr(data_train_wss.squeeze('spec'),annAv_ersst,dim='year')
corrs_wss.persist()

corrs_acc=xr.corr(data_train_acc.squeeze('spec'),annAv_ersst,dim='year')
corrs_acc.persist()

NameError: name 'data_train_sss' is not defined

In [16]:
ssta_pred, corrs, ssta_reco = xr.apply_ufunc(new_linreg,
                                             data_train,
                                             annAv_ersst,
                                             all_data,
                                             input_core_dims=[['year', 'spec'], ['year'], ['time', 'spec']],
                                             # output_core_dims=[[],[],[],[]],
                                             #output_sizes={'bound':2},
                                             # output_core_dims=[['year'],[],['bound'],['lyear']],
                                             output_core_dims=[['year'], [], ['time']],
                                             output_dtypes=[np.float32, np.float32, np.float32],
                                             vectorize=True,
                                             dask='parallelized',
                                             )

ssta_pred_sss, corrs_sss, ssta_reco_sss = xr.apply_ufunc(new_linreg,
                                                         data_train_sss,
                                                         annAv_ersst,
                                                         all_data_sss,
                                                         input_core_dims=[['year', 'spec'], ['year'], ['time', 'spec']],
                                                         # output_core_dims=[[],[],[],[]],
                                                         #output_sizes={'bound':2},
                                                         # output_core_dims=[['year'],[],['bound'],['lyear']],
                                                         output_core_dims=[['year'], [], ['time']],
                                                         output_dtypes=[np.float32, np.float32, np.float32],
                                                         vectorize=True,
                                                         dask='parallelized',
                                                         )

ssta_pred_sss_wss, corrs_sss_wss, ssta_reco_sss_wss = xr.apply_ufunc(new_linreg,
                                                                     data_train_sss_wss,
                                                                     annAv_ersst,
                                                                     all_data_sss_wss,
                                                                     input_core_dims=[['year', 'spec'], ['year'],
                                                                                      ['time', 'spec']],
                                                                     # output_core_dims=[[],[],[],[]],
                                                                     #output_sizes={'bound':2},
                                                                     # output_core_dims=[['year'],[],['bound'],['lyear']],
                                                                     output_core_dims=[['year'], [], ['time']],
                                                                     output_dtypes=[np.float32, np.float32, np.float32],
                                                                     vectorize=True,
                                                                     dask='parallelized',
                                                                     )

ssta_pred = ssta_pred.persist()
corrs = corrs.persist()
ssta_reco = ssta_reco.persist()


NameError: name 'data_train' is not defined

In [17]:
def bootstraps(ttarget, ypred,corr_coeff):
    #need to adapt this and the return vals

    #should this be bigger, perhaps 13? Seems to be an inconsistency in the different versions.
    bs = CircularBlockBootstrap(13, years,x=ttarget,y=ypred) #checked autocorrelation length of the data, 6 is consistant with block length chosen by Mudelsee code

    if np.isnan(corr_coeff):
        conf_interval=np.zeros(2)
    else:
        conf_interval=bs.conf_int(pearson,reps=2000,method='bca').flatten()

    return conf_interval

In [18]:
conf_int = xr.apply_ufunc(bootstraps,
                          annAv_ersst,
                          ssta_pred,
                          corrs,
                          input_core_dims=[['year'], ['year'], []],
                          # output_core_dims=[[],[],[],[]],
                          output_sizes={'bound': 2},
                          # output_core_dims=[['year'],[],['bound'],['lyear']],
                          output_core_dims=[['bound']],
                          output_dtypes=[np.float32],
                          vectorize=True,
                          dask='parallelized',
                          )
conf_int_sss = xr.apply_ufunc(bootstraps,
                              annAv_ersst,
                              ssta_pred_sss,
                              corrs_sss,
                              input_core_dims=[['year'], ['year'], []],
                              # output_core_dims=[[],[],[],[]],
                              output_sizes={'bound': 2},
                              # output_core_dims=[['year'],[],['bound'],['lyear']],
                              output_core_dims=[['bound']],
                              output_dtypes=[np.float32],
                              vectorize=True,
                              dask='parallelized',
                              )
conf_int_sss_wss = xr.apply_ufunc(bootstraps,
                                  annAv_ersst,
                                  ssta_pred_sss_wss,
                                  corrs_sss_wss,
                                  input_core_dims=[['year'], ['year'], []],
                                  # output_core_dims=[[],[],[],[]],
                                  output_sizes={'bound': 2},
                                  # output_core_dims=[['year'],[],['bound'],['lyear']],
                                  output_core_dims=[['bound']],
                                  output_dtypes=[np.float32],
                                  vectorize=True,
                                  dask='parallelized',
                                  )

NameError: name 'ssta_pred' is not defined

In [ ]:

ds = conf_int.transpose('bound', 'lat', 'lon').to_dataset(name='conf_interval')
ds.to_netcdf("outputs/unsmoothed/tmp/ssta_cis.nc", mode='w')

ds = conf_int_sss.transpose('bound', 'lat', 'lon').to_dataset(name='conf_interval')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_sss_cis.nc", mode='w')
ds = conf_int_sss_wss.transpose('bound', 'lat', 'lon').to_dataset(name='conf_interval')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_sss_wss_cis.nc", mode='w')
ds = ssta_reco.transpose('time', 'lat', 'lon').to_dataset(name='ssta_reconstructed')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_reco.nc", mode='w')
ds = ssta_reco_sss.transpose('time', 'lat', 'lon').to_dataset(name='ssta_reconstructed')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_reco_sss.nc", mode='w')
ds = ssta_reco_sss_wss.transpose('time', 'lat', 'lon').to_dataset(name='ssta_reconstructed')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_reco_sss_wss.nc", mode='w')

ds = ssta_pred.transpose('year', 'lat', 'lon').to_dataset(name='ssta')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_pred.nc", mode='w')
ds = ssta_pred_sss.transpose('year', 'lat', 'lon').to_dataset(name='ssta')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_pred_sss.nc", mode='w')
ds = ssta_pred_sss_wss.transpose('year', 'lat', 'lon').to_dataset(name='ssta')
ds.to_netcdf("new_outputs/unsmoothed/tmp/ssta_pred_sss_wss.nc", mode='w')
ds = corrs.to_dataset(name='corr_coefficient')
ds.to_netcdf("new_outputs/unsmoothed/tmp/corr_coeff.nc", mode='w')
ds = corrs_sss.to_dataset(name='corr_coefficient')
ds.to_netcdf("new_outputs/unsmoothed/tmp/corr_coeff_sss.nc", mode='w')
ds = corrs_sss_wss.to_dataset(name='corr_coefficient')
ds.to_netcdf("new_outputs/unsmoothed/tmp/corr_coeff_sss_wss.nc", mode='w')